## Task 1

In [3]:
import pandas as pd
import re
from thefuzz import fuzz, process
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from sklearn.feature_extraction import text
import numpy as np

In [4]:
new_q = pd.read_csv("new_queries.csv")
resolved_q = pd.read_csv("resolved_queries.csv")

print("Unresolved queries sample:\n", new_q.head(), "\n")
print("Resolved queries sample:\n", resolved_q.head())

Unresolved queries sample:
                              Variation_Query  Matches_With_Query_ID
0           Unabel to conect to the internet                      1
1                  Can’t connect to internet                      1
2                        Intenet not working                      1
3               Payment failed while chekout                      2
4  Payment did not go through during chckout                      2 

Resolved queries sample:
    Query_ID                    Pre_Resolved_Query
0         1     Unable to connect to the internet
1         2        Payment failed during checkout
2         3     App crashes when opening settings
3         4   Forgot password and unable to reset
4         5  Unable to upload files to the server


In [5]:
stopwords = text.ENGLISH_STOP_WORDS

def clean_text(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(f"[{string.punctuation}]", " ", s)
    s = " ".join([w for w in s.split() if w not in stopwords])
    return s.strip()

new_q["clean"] = new_q["Variation_Query"].map(clean_text)
resolved_q["clean"] = resolved_q["Pre_Resolved_Query"].map(clean_text)

In [6]:
methods = {
    "ratio": fuzz.ratio,
    "partial_ratio": fuzz.partial_ratio,
    "token_sort_ratio": fuzz.token_sort_ratio,
    "token_set_ratio": fuzz.token_set_ratio,
}

choices = resolved_q["clean"].tolist()

fuzzy_results = []

for query in new_q["clean"]:
    row = {"query": query}
    for name, scorer in methods.items():
        match = process.extractOne(query, choices, scorer=scorer)
        row[name] = match[1]   # store similarity score
    fuzzy_results.append(row)

fuzzy_df = pd.DataFrame(fuzzy_results)
fuzzy_df

,query,ratio,partial_ratio,token_sort_ratio,token_set_ratio
0,unabel conect internet,93,91,93,93
1,can’t connect internet,80,87,76,84
2,intenet working,42,61,47,47
3,payment failed chekout,98,95,98,98
4,payment did chckout,86,74,86,86
5,payment issue check,76,79,76,76
6,application crashes opening setings,86,92,86,86
7,app crash going settings,88,79,88,88
8,settings cause app chrash,34,53,68,68
9,forgot passwrd reset,83,83,83,83


In [7]:
fuzzy_df.describe()


,ratio,partial_ratio,token_sort_ratio,token_set_ratio
count,20.000000,20.000000,20.000000,20.00000
mean,70.350000,76.200000,77.950000,82.05000
std,21.221327,14.645279,13.539319,13.37112
min,34.000000,51.000000,47.000000,47.00000
25%,50.750000,62.750000,67.750000,74.75000
50%,78.000000,77.500000,79.000000,83.50000
75%,86.500000,89.500000,86.500000,93.50000
max,98.000000,98.000000,98.000000,100.00000


In [8]:
best_method = "token_set_ratio"
scores = fuzzy_df[best_method]

print("Score Quantiles:\n", scores.quantile([0.5, 0.75, 0.9, 0.95, 0.99]))
threshold = int(scores.quantile(0.90))
print("Chosen threshold:", threshold)

Score Quantiles:
 0.50    83.50
0.75    93.50
0.90    96.20
0.95    98.10
0.99    99.62
Name: token_set_ratio, dtype: float64
Chosen threshold: 96


In [9]:
final_fuzzy = []

for query in new_q["clean"]:
    match = process.extractOne(query, choices, scorer=fuzz.token_set_ratio)
    text, score = match[0], match[1]
    idx = resolved_q[resolved_q["clean"] == text].index[0]
    qid = resolved_q.iloc[idx]["Query_ID"]

    if score >= threshold:
        final_fuzzy.append([query, text, qid, score, "Accepted"])
    else:
        final_fuzzy.append([query, text, qid, score, "Low Score"])

final_fuzzy_df = pd.DataFrame(final_fuzzy,
                              columns=["Query", "Matched", "Query_ID", "Score", "Status"])
final_fuzzy_df.head()

,Query,Matched,Query_ID,Score,Status
0,unabel conect internet,unable connect internet,1,93,Low Score
1,can’t connect internet,unable connect internet,1,84,Low Score
2,intenet working,unable connect internet,1,47,Low Score
3,payment failed chekout,payment failed checkout,2,98,Accepted
4,payment did chckout,payment failed checkout,2,86,Low Score


In [10]:
count_vec = CountVectorizer()
X = count_vec.fit_transform(new_q["clean"].tolist() + resolved_q["clean"].tolist())

n_new = len(new_q)
X_new, X_res = X[:n_new], X[n_new:]

bow_results = []
for i, q in enumerate(new_q["clean"]):
    sims = cosine_similarity(X_new[i], X_res).flatten()
    idx = sims.argmax()
    best_score = sims[idx]
    best_text = resolved_q.iloc[idx]["clean"]
    best_id = resolved_q.iloc[idx]["Query_ID"]
    bow_results.append([q, best_text, best_id, best_score])

bow_df = pd.DataFrame(bow_results, columns=["Query", "BoW_Match", "BoW_ID", "BoW_Score"])
bow_df.head()

,Query,BoW_Match,BoW_ID,BoW_Score
0,unabel conect internet,unable connect internet,1,0.333333
1,can’t connect internet,unable connect internet,1,0.666667
2,intenet working,unable connect internet,1,0.000000
3,payment failed chekout,payment failed checkout,2,0.666667
4,payment did chckout,payment failed checkout,2,0.333333


In [11]:
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(new_q["clean"].tolist() + resolved_q["clean"].tolist())

X_new, X_res = X[:n_new], X[n_new:]

tfidf_results = []
for i, q in enumerate(new_q["clean"]):
    sims = cosine_similarity(X_new[i], X_res).flatten()
    idx = sims.argmax()
    best_score = sims[idx]
    best_text = resolved_q.iloc[idx]["clean"]
    best_id = resolved_q.iloc[idx]["Query_ID"]
    tfidf_results.append([q, best_text, best_id, best_score])

tfidf_df = pd.DataFrame(tfidf_results, columns=["Query", "TFIDF_Match", "TFIDF_ID", "TFIDF_Score"])
tfidf_df.head()

,Query,TFIDF_Match,TFIDF_ID,TFIDF_Score
0,unabel conect internet,unable connect internet,1,0.256410
1,can’t connect internet,unable connect internet,1,0.681156
2,intenet working,unable connect internet,1,0.000000
3,payment failed chekout,payment failed checkout,2,0.558907
4,payment did chckout,payment failed checkout,2,0.219802


In [12]:
comparison = pd.concat([
    final_fuzzy_df[["Query", "Matched", "Query_ID", "Score"]].rename(columns={"Matched":"Fuzzy_Match", "Score":"Fuzzy_Score"}),
    bow_df[["BoW_Match", "BoW_ID", "BoW_Score"]],
    tfidf_df[["TFIDF_Match", "TFIDF_ID", "TFIDF_Score"]],
], axis=1)

comparison.head(20)

,Query,Fuzzy_Match,Query_ID,Fuzzy_Score,BoW_Match,BoW_ID,BoW_Score,TFIDF_Match,TFIDF_ID,TFIDF_Score
0,unabel conect internet,unable connect internet,1,93,unable connect internet,1,0.333333,unable connect internet,1,0.256410
1,can’t connect internet,unable connect internet,1,84,unable connect internet,1,0.666667,unable connect internet,1,0.681156
2,intenet working,unable connect internet,1,47,unable connect internet,1,0.000000,unable connect internet,1,0.000000
3,payment failed chekout,payment failed checkout,2,98,payment failed checkout,2,0.666667,payment failed checkout,2,0.558907
4,payment did chckout,payment failed checkout,2,86,payment failed checkout,2,0.333333,payment failed checkout,2,0.219802
5,payment issue check,payment failed checkout,2,76,payment failed checkout,2,0.333333,payment failed checkout,2,0.219802
6,application crashes opening setings,app crashes opening settings,3,86,app crashes opening settings,3,0.500000,app crashes opening settings,3,0.485807
7,app crash going settings,app crashes opening settings,3,88,app crashes opening settings,3,0.500000,app crashes opening settings,3,0.393307
8,settings cause app chrash,app crashes opening settings,3,68,app crashes opening settings,3,0.500000,app crashes opening settings,3,0.393307
9,forgot passwrd reset,forgot password unable reset,4,83,forgot password unable reset,4,0.577350,forgot password unable reset,4,0.554582


## Task 2

In [13]:
import re
import pandas as pd
from rapidfuzz import fuzz, process
from sklearn.metrics import accuracy_score

In [14]:
base = pd.read_csv("base_names.csv")
vari = pd.read_csv("name_variations.csv")

In [15]:
def normalize(n):
    if pd.isna(n):
        return ""
    n = str(n).strip().lower()
    if "," in n:
        parts = [p.strip() for p in n.split(",", 1)]
        if len(parts) == 2:
            n = parts[1] + " " + parts[0]
    n = re.sub(r"[^a-z0-9\s]", " ", n)
    n = re.sub(r"\s+", " ", n).strip()
    return n

base["norm"] = base["Base_Name"].apply(normalize)
vari["norm"] = vari["Variation"].apply(normalize)

In [16]:
rows = []
for v_show, v_norm in zip(vari["Variation"], vari["norm"]):
    candidates = process.extract(
        v_norm,
        base["norm"].tolist(),
        scorer=fuzz.token_set_ratio,
        limit=3
    )
    if candidates:
        best_norm, best_score, best_idx = candidates[0]
        best_name = base.iloc[best_idx]["Base_Name"]
    else:
        best_name, best_score, candidates = "No Match", 0, []

    top_names = [base.iloc[idx]["Base_Name"] for (_, _, idx) in candidates]
    top_scores = [score for (_, score, _) in candidates]

    rows.append([v_show, best_name, best_score, top_names, top_scores])

In [17]:
out = pd.DataFrame(rows, columns=[
    "Variation", "Best_Match", "Best_Score", "Top3_Names", "Top3_Scores"
])

print(out.head(5))

      Variation    Best_Match  Best_Score  \
0  Thomas  King   Thomas King  100.000000   
1    ThomasKing   Thomas King   57.142857   
2  Maria Garcia  Maria Garcia  100.000000   
3     MaryLewis    Mary Lewis   52.631579   
4      Nancy W.  Nancy Wright   83.333333   

                                     Top3_Names  \
0         [Thomas King, John Smith, Mary Lewis]   
1  [Thomas King, Linda Johnson, David Martinez]   
2    [Maria Garcia, David Martinez, Mary Lewis]   
3  [Mary Lewis, Elizabeth Wilson, Maria Garcia]   
4  [Nancy Wright, Paul Allen, Michael O'Connor]   

                                         Top3_Scores  
0    [100.0, 38.095238095238095, 38.095238095238095]  
1  [57.142857142857146, 34.78260869565217, 33.333...  
2      [100.0, 53.84615384615385, 45.45454545454545]  
3      [52.63157894736842, 40.0, 38.095238095238095]  
4  [83.33333333333333, 35.294117647058826, 34.782...  


In [18]:
if "Matches_With_Base_Name" in vari.columns:
    eval_df = vari[["Variation", "Matches_With_Base_Name"]].merge(
        out[["Variation", "Best_Match"]],
        on="Variation",
        how="left"
    )
    eval_df = eval_df.fillna("No Match")

    acc = accuracy_score(eval_df["Matches_With_Base_Name"], eval_df["Best_Match"])
    print("Accuracy:", round(acc, 4))

Accuracy: 1.0
